### **Autores**

JUAN DAVID GARCIA MEJIA ID UAO: 2233216

JOHAN DAVID ROMERO RODRIGUEZ ID UAO:2235517

EDWIN JAVIER CHAPARRO ARBOLEDA ID UAO: 2227299

### **Introducción:**

En este trabajo, se aborda el problema de la clasificación de movimientos a partir de datos de un acelerómetro. La capacidad de reconocer e identificar diferentes movimientos o actividades físicas tiene aplicaciones en campos como el monitoreo de la actividad física, la rehabilitación y el entrenamiento deportivo. El objetivo principal es desarrollar modelos de aprendizaje automático capaces de clasificar correctamente los movimientos a partir de los datos del acelerómetro

### **Marco teórico:**

El reconocimiento de movimientos a partir de datos de acelerómetro se enmarca dentro del campo del aprendizaje automático y, más específicamente, del aprendizaje profundo. Se emplean técnicas como las redes neuronales convolucionales (CNN), las redes neuronales recurrentes (RNN) y las redes neuronales feed-forward (MLP) para aprender patrones y características relevantes de los datos de acelerómetro y realizar la clasificación de movimientos.



### **Descripción del problema a solucionar:**

El problema consiste en tomar los datos de un acelerómetro, que representan una secuencia de mediciones tridimensionales (x, y, z) a lo largo del tiempo, y clasificarlos en una de las siguientes cinco clases de movimientos: Bicep Curl, Chest Fly, Front Raise, Lateral Raise y Reverse Fly. Estos datos de acelerómetro provienen de dispositivos vestibles utilizados durante la realización de ejercicios de entrenamiento.



### **Planteamiento de la solución:**

Para abordar este problema, se plantea el desarrollo y evaluación de tres modelos de aprendizaje profundo: una red neuronal feed-forward (MLP), una red neuronal convolucional 1D (Conv1D) y una red neuronal recurrente (RNN). Estos modelos se entrenan con un conjunto de datos de entrenamiento y se evalúan en un conjunto de datos de prueba. Se realiza un preprocesamiento de los datos, incluyendo la estandarización y la segmentación en patrones de entrada adecuados para cada modelo.

In [0]:

from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow import keras

In [0]:
# Ejemplo de la carga de un archivo *.json con las mediciones de un acelerómetro
RutaFile="/content/drive/MyDrive/Colab Notebooks/Datos/Acelerometro/training/Chest_Fly.4r90g5ml.ingestion-848d69cd95-lzk2f.json"
dataframe = pd.read_json(RutaFile)
print(dataframe.to_string())
Valores = dataframe.iloc[:,:].values
print(Valores.shape)
print(Valores)
print(Valores[7,2][0:-1])

DatosOri1=Valores[7,2][0:-1]
Datos1=np.array(DatosOri1)
print(Datos1.shape)
plt.plot(Datos1)

In [0]:
# Función para cargar datos de un directorio
def cargar_datos(directory):
    # Obtener la lista de archivos en el directorio y ordenarlos alfabéticamente
    files = os.listdir(directory)
    files.sort()

    # Lista para almacenar los datos de todos los archivos
    datos = []

    # Leer datos de cada archivo y agregarlos a la lista
    for file in files:
        # Leer el archivo JSON utilizando Pandas
        df = pd.read_json(os.path.join(directory, file))
        # Extraer los datos de la columna específica del DataFrame
        datos.extend(df.iloc[7, 2][0:-1])

    # Convertir la lista de datos a un arreglo numpy
    datos = np.array(datos)

    return datos

# Función para preprocesar los datos para MLP y Conv1D
def preprocesar_datos_mlp_conv(datos, scaler):
    # Escalar los datos utilizando el objeto scaler proporcionado
    datos_esc = scaler.transform(datos)
    # Definir el tamaño de la muestra para la entrada del modelo (104 en este caso)
    tamano_muestra = 104
    # Calcular el número de patrones en los datos escalados
    num_patrones = len(datos_esc) // tamano_muestra
    # Reorganizar los datos en el formato adecuado para MLP o Conv1D
    X = datos_esc[:num_patrones*tamano_muestra].reshape(num_patrones, tamano_muestra, 3)
    return X

# Función para preprocesar los datos para RNN
def preprocesar_datos_rnn(datos, scaler):
    # No hay diferencia en el preprocesamiento entre RNN y MLP/Conv1D, se reutiliza la función existente
    return preprocesar_datos_mlp_conv(datos, scaler)


In [0]:
# Directorios de datos
training_directory = '/content/drive/MyDrive/Colab Notebooks/Datos/Acelerometro/training/'
testing_directory = '/content/drive/MyDrive/Colab Notebooks/Datos/Acelerometro/testing/'

# Cargar datos de entrenamiento y prueba
datos_entrenamiento = cargar_datos(training_directory)
datos_prueba = cargar_datos(testing_directory)

# Escalar datos
scaler = StandardScaler()
scaler.fit(datos_entrenamiento)
# Preprocesar datos para MLP y Conv1D
X_entrenamiento_mlp_conv = preprocesar_datos_mlp_conv(datos_entrenamiento, scaler)
X_prueba_mlp_conv = preprocesar_datos_mlp_conv(datos_prueba, scaler)
# Preprocesar datos para RNN
X_entrenamiento_rnn = preprocesar_datos_rnn(datos_entrenamiento, scaler)
X_prueba_rnn = preprocesar_datos_rnn(datos_prueba, scaler)

# Definir etiquetas de entrenamiento y prueba
y_entrenamiento = np.repeat(np.arange(5), len(X_entrenamiento_mlp_conv) // 5)
y_prueba = np.repeat(np.arange(5), len(X_prueba_mlp_conv) // 5)

# Ajustar el número de etiquetas de prueba para que coincida con el número total de muestras de prueba
num_muestras_faltantes = len(X_prueba_mlp_conv) - len(y_prueba)
y_prueba = np.concatenate([y_prueba, np.zeros(num_muestras_faltantes)], axis=0)

# Convertir etiquetas a formato one-hot
Y_entrenamiento = keras.utils.to_categorical(y_entrenamiento)
Y_prueba = keras.utils.to_categorical(y_prueba)

# Imprimir dimensiones de los datos
print("Datos Entrenamiento: ", datos_entrenamiento.shape)
print("Datos Test: ", datos_prueba.shape)
print(X_prueba_mlp_conv.shape)
print(X_prueba_rnn.shape)
print(Y_prueba.shape)


In [0]:
modelos = {
    "MLP": keras.Sequential([
        keras.layers.Flatten(input_shape=(tamano_muestra, 3)),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(32, activation="relu"),
        keras.layers.Dense(5, activation="softmax"),  # Corregido a 5 neuronas para 5 clases
    ]),
    "Conv1D": keras.Sequential([
        keras.layers.Conv1D(32, 3, activation="relu", input_shape=(tamano_muestra, 3)),
        keras.layers.MaxPooling1D(),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(5, activation="softmax"),  # Corregido a 5 neuronas para 5 clases
    ]),
    "RNN": keras.Sequential([
        keras.layers.LSTM(32, input_shape=(tamano_muestra, 3)),
        keras.layers.Dense(5, activation="softmax"),  # Corregido a 5 neuronas para 5 clases
    ]),
}


Preprocesamiento de Datos En esta fase cargaremos los datos, los dividiremos en conjuntos de entrenamiento y prueba, y los escalaremos si es necesario.**bold text**

In [0]:
# Visualización de los datos de entrenamiento (Acelerómetro)
plt.figure(figsize=(10, 6))
plt.plot(datos_entrenamiento)
plt.title('Datos de entrenamiento (Acelerómetro)')
plt.xlabel('Muestras')
plt.ylabel('Valor')
plt.show()

# Visualización de los primeros 100 datos de entrenamiento sin escalar
plt.figure(figsize=(10, 6))
plt.plot(datos_entrenamiento[:600])
plt.title('Datos de entrenamiento sin escalar')
plt.xlabel('Muestras')
plt.ylabel('Valor')
plt.show()

# Visualización de los primeros 100 datos de entrenamiento escalados
plt.figure(figsize=(10, 6))
plt.plot(scaler.transform(datos_entrenamiento[:600]))
plt.title('Datos de entrenamiento escalados')
plt.xlabel('Muestras')
plt.ylabel('Valor')
plt.show()

# Visualización de un patrón de entrenamiento (ejemplo)
plt.figure(figsize=(10, 6))
plt.plot(X_entrenamiento_mlp_conv[0])
plt.title('Patrón de entrenamiento (ejemplo)')
plt.xlabel('Muestras')
plt.ylabel('Valor escalado')
plt.show()


Fase 2: Definición y Entrenamiento de Modelos
Aquí definiremos los modelos MLP, Conv1D y RNN, los entrenaremos con los datos de entrenamiento y registraremos sus historias de entrenamiento.

In [0]:
# Entrenamiento y generación de esquemas
historias = {}
for nombre, modelo in modelos.items():
    print(f"Entrenando y generando esquema para el modelo {nombre}")
    modelo.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    modelo.summary()  # Mostrar resumen del modelo
    historia = modelo.fit(X_entrenamiento_mlp_conv if nombre in ["MLP", "Conv1D"] else X_entrenamiento_rnn,
                           Y_entrenamiento,
                           epochs=100,
                           batch_size=None if nombre == "RNN" else 32,
                           verbose=0)
    historias[nombre] = historia

In [0]:
# Gráfica de la evolución de la pérdida durante el entrenamiento
plt.figure(figsize=(10, 6))
for nombre, historia in historias.items():
    plt.plot(historia.history['loss'], label=nombre)
plt.title('Evolución de la pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

# Gráficas de la evolución de la pérdida durante el entrenamiento para cada modelo
for nombre, historia in historias.items():
    plt.figure(figsize=(10, 6))
    plt.plot(historia.history['loss'])
    plt.title(f'Evolución de la pérdida durante el entrenamiento - {nombre}')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.show()



In [0]:
# Evaluación de modelos y visualización de la matriz de confusión
for nombre, modelo in modelos.items():
    print(f"Modelo: {nombre}")
    resultados = modelo.evaluate(X_prueba_mlp_conv if nombre in ["MLP", "Conv1D"] else X_prueba_rnn,
                                  Y_prueba,
                                  verbose=0)
    print("Pérdida:", resultados[0])
    print("Precisión:", resultados[1])

    # Predicciones y matriz de confusión
    predicciones = modelo.predict(X_prueba_mlp_conv if nombre in ["MLP", "Conv1D"] else X_prueba_rnn)
    y_pred_class = np.argmax(predicciones, axis=1)
    print("Matriz de confusión:")
    cm = confusion_matrix(y_prueba, y_pred_class)
    print(cm)
    print("Reporte de clasificación:")
    print(classification_report(y_prueba, y_pred_class))

    # Gráfica de la matriz de confusión
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title(f'Matriz de Confusión - {nombre}')
    plt.xlabel('Predicciones')
    plt.ylabel('Valores verdaderos')
    plt.show()


In [0]:
# Evaluación de modelos y selección del mejor modelo
mejor_modelo = None
mejor_precision = 0

for nombre, modelo in modelos.items():
    # Entrenar el modelo y guardar la historia
    historia = modelo.fit(X_entrenamiento_mlp_conv if nombre in ["MLP", "Conv1D"] else X_entrenamiento_rnn,
                           Y_entrenamiento,
                           epochs=100,
                           batch_size=None if nombre == "RNN" else 32,
                           verbose=0)

    # Evaluar el modelo en el conjunto de prueba
    resultados = modelo.evaluate(X_prueba_mlp_conv if nombre in ["MLP", "Conv1D"] else X_prueba_rnn,
                                  Y_prueba,
                                  verbose=0)
    perdida = resultados[0]
    precision = resultados[1]

    # Imprimir los resultados
    print(f"Modelo: {nombre}")
    print("Pérdida:", perdida)
    print("Precisión:", precision)

    # Comparar con el mejor modelo actual
    if precision > mejor_precision:
        mejor_modelo = nombre
        mejor_precision = precision

# Imprimir el mejor modelo y su precisión
print(f"El mejor modelo es {mejor_modelo} con una precisión de {mejor_precision}.")


In [0]:
import tensorflow as tf

# Código para crear y compilar el modelo
modelo = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=(104, 3)),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Conv1D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Conv1D(64, 3, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')  # El número de clases es 5 según el contexto
])

modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
# ...

# Guardar el modelo en formato h5
modelo.save('modelo.h5')


In [0]:
import tensorflow as tf
import numpy as np

# Cargar el modelo
modelo = tf.keras.models.load_model('modelo.h5')

# Función para cargar y preprocesar los datos de un archivo JSON
def cargar_datos_json(archivo_json):
    with open(archivo_json, 'r') as f:
        datos_json = json.load(f)

    # Extraer los valores del "payload"
    valores = datos_json['payload']['values']

    # Convertir los valores a un arreglo numpy
    datos_acelerometro = np.array(valores)

    return datos_acelerometro


# Función para inferir la clase a partir de las predicciones
def inferir_clase(prediccion):
    clases = ['Bicep_Curl', 'Chest_Fly', 'Front_Raise', 'Lateral_Raise', 'reverse_Fly']
    indice_clase = np.argmax(prediccion)
    return clases[indice_clase]

# Ruta al archivo JSON que contiene los datos del acelerómetro
archivo_json = '/content/Bicep_Curl.4r90jnsk.ingestion-848d69cd95-ttl72.json'

# Cargar y preprocesar los datos
datos_acelerometro = cargar_datos_json(archivo_json)

# Asegurarnos de que los datos tienen al menos 104 muestras
if datos_acelerometro.shape[0] < 104:
    print("El archivo JSON no tiene suficientes muestras para realizar una predicción.")
else:
    # Tomar solo las primeras 104 muestras (o ajustar según sea necesario)
    datos_acelerometro = datos_acelerometro[:104]

    # Realizar la predicción
    prediccion = modelo.predict(datos_acelerometro.reshape(1, 104, 3))  # Reshape para que coincida con la forma esperada por el modelo

    # Obtener la clase predicha
    clase_predicha = inferir_clase(prediccion)
    print("Clase predicha:", clase_predicha)


### **Resultados:**

En el código, se entrenan y evalúan tres modelos de aprendizaje profundo: una red neuronal feed-forward (MLP), una red neuronal convolucional 1D (Conv1D) y una red neuronal recurrente (RNN). Los modelos se entrenan con un conjunto de datos de entrenamiento y se evalúan en un conjunto de datos de prueba.

Se muestran las métricas de rendimiento, como la pérdida y la precisión, para cada modelo en el conjunto de prueba:

MLP:
Pérdida: 2.369173288345337
Precisión: 0.7627118825912476

Conv1D:
Pérdida: 2.7176055908203125
Precisión: 0.7627118825912476

RNN:
Pérdida: 1.602267861366272
Precisión: 0.7288135886192322

Además, se calculan las matrices de confusión y los reportes de clasificación para cada modelo, lo que permite evaluar el rendimiento en términos de precisión, recall y F1-score para cada clase.

Finalmente, se selecciona el mejor modelo basado en la precisión en el conjunto de prueba, siendo el modelo MLP el que obtuvo la mayor precisión de 0.7627118825912476.

### **Conclusiones:**

Los resultados obtenidos muestran que los tres modelos evaluados (MLP, Conv1D y RNN) lograron un rendimiento similar en la clasificación de movimientos a partir de los datos del acelerómetro. Sin embargo, el modelo MLP tuvo un ligero mejor desempeño en términos de precisión en el conjunto de prueba.

Es importante mencionar que los modelos podrían mejorarse aún más mediante la optimización de hiperparámetros, como el número de épocas de entrenamiento, el tamaño de lote y las tasas de aprendizaje. Además, se podrían explorar arquitecturas más complejas o técnicas adicionales, como la regularización o el aumento de datos, para mejorar el rendimiento y la capacidad de generalización de los modelos.

Una limitación importante del enfoque actual es que los modelos se entrenan y evalúan utilizando un conjunto de datos único. Para obtener resultados más robustos y confiables, sería recomendable evaluar los modelos en múltiples conjuntos de datos, posiblemente recopilados en diferentes entornos y condiciones.

En general, este trabajo demuestra el potencial del aprendizaje profundo para abordar el problema de la clasificación de movimientos a partir de datos de acelerómetro, pero también resalta la necesidad de continuar explorando y mejorando los enfoques utilizados.

### **Referencias:**

Brownlee, J. (2020). Deep Learning for Time Series Forecasting. Machine Learning Mastery.

Zhang, M., Sawchuk, A. A. (2012). Human Daily Activity Recognition With Sparse Representation Using Wearable Sensors. IEEE Journal of Biomedical and Health Informatics, 16(3), 553-560.

Ronao, C. A., Cho, S. B. (2016). Human activity recognition with smartphone sensors using deep learning neural networks. Expert Systems with Applications, 59, 235-244.

Tensorflow: https://www.tensorflow.org/

